# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with 'mongoimport --db uk_food --collection establishments --file "C:\Users\Mrod9\downloads\challenge 12 instructions\starter_code\resources\establishments.json" --jsonArray'


In [4]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [5]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [7]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [8]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [9]:
# review the collections in our new database


In [10]:
# assign the collection to a variable
establishments = db['establishments']

In [11]:
# review a document in the establishments collection
sample_document = establishments.find_one()

In [12]:
# Print the sample document
print("\nSample document from the 'establishments' collection:")
pprint(sample_document)


Sample document from the 'establishments' collection:
{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677f400fbb71f9e266275f15'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel'

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [15]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant",
    "CuisineType": "Halal",
    "RatingValue": None,
    "Location": "Greenwich",
}

In [16]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)

In [17]:
# Check that the new restaurant was inserted
inserted_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})

print(inserted_restaurant)

{'_id': ObjectId('677f402ba00aab0d3abc8cc5'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant', 'CuisineType': 'Halal', 'RatingValue': None, 'Location': 'Greenwich'}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [19]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# Query to find the BusinessTypeID for "Restaurant/Cafe/Canteen"
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Define the projection to return only BusinessTypeID and BusinessType fields
projection = {"BusinessTypeID": 1, "BusinessType": 1}

# Find the document matching the query with the specified projection
business_type = establishments.find_one(query, projection)

# Print the result
if business_type:
    pprint(business_type)
else:
    print("No matching business type found.")


{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('677f400fbb71f9e266275f16')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [21]:
# Update the new restaurant with the correct BusinessTypeID
# Step 1: Find the BusinessTypeID for "Restaurant/Cafe/Canteen"
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "_id": 0}  # Only return BusinessTypeID
business_type = establishments.find_one(query, projection)

# Check if we found the BusinessTypeID
if business_type:
    business_type_id = business_type["BusinessTypeID"]
    print(f"Found BusinessTypeID: {business_type_id}")
    
    # Step 2: Update the new restaurant with the correct BusinessTypeID
    update_query = {"BusinessName": "Mex Flavours"}
    update_data = {"$set": {"BusinessTypeID": business_type_id}}

    # Perform the update
    update_result = establishments.update_one(update_query, update_data)


Found BusinessTypeID: 1


In [22]:
# Suppose this is your code where you search for the BusinessTypeID
business_type_id = "Restaurant/Cafe/Canteen"
business_type_found = establishments.find_one({"BusinessType": business_type_id})

# Now check if the BusinessTypeID was found
if business_type_found:
    # If found, update the restaurant
    update_result = establishments.update_one(
        {"BusinessType": business_type_id},
        {"$set": {"RestaurantName": "Penang Flavours"}}
    )

    # Confirm if the restaurant was updated
    if update_result.modified_count > 0:
        print("Restaurant updated successfully.")
    else:
        print("No changes made to the restaurant.")
else:
    print ("No matching BusinessTypeID found for 'Restaurant/Cafe/Canteen'.")

print

Restaurant updated successfully.


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [24]:
# Find how many documents have LocalAuthorityName as "Dover"
count_dover = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of establishments in Dover: {count_dover}")

Number of establishments in Dover: 0


In [25]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"Number of establishments removed: {delete_result.deleted_count}")

Number of establishments removed: 0


In [26]:
# Check if any remaining documents include Dover
remaining_dover = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Remaining establishments in Dover: {remaining_dover}")

Remaining establishments in Dover: 0


In [27]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()
print("A remaining document:", remaining_document)

A remaining document: {'_id': ObjectId('677f400fbb71f9e266275f15'), 'FHRSID': 647177, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000041489', 'BusinessName': 'Wear Bay Bowls Club', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': 'Wear Bay Road', 'AddressLine2': 'Folkestone', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT19 6PY', 'Phone': '', 'RatingValue': 4, 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2014-03-31T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.196408', 'latitude': '51.086058'}, 'RightToReply': '', 'Distance': 4591.821311183521, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount'

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [30]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {"Latitude": {"$type": "string"}},
    [{"$set": {"Latitude": {"$toDecimal": "$Latitude"}}}]
)

establishments.update_many(
    {"Longitude": {"$type": "string"}},
    [{"$set": {"Longitude": {"$toDecimal": "$Longitude"}}}]
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [32]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [33]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [34]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one()
print(sample_document)

# Print types of Latitude, Longitude, and RatingValue
if 'Latitude' in sample_document:
    print(f"Latitude type: {type(sample_document['Latitude'])}")
else:
    print("Latitude field is not present in this document.")

if 'Longitude' in sample_document:
    print(f"Longitude type: {type(sample_document['Longitude'])}")
else:
    print("Longitude field is not present in this document.")

if 'RatingValue' in sample_document:
    print(f"RatingValue type: {type(sample_document['RatingValue'])}")
else:
    print("RatingValue field is not present in this document.")

{'_id': ObjectId('677f400fbb71f9e266275f15'), 'FHRSID': 647177, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000041489', 'BusinessName': 'Wear Bay Bowls Club', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': 'Wear Bay Road', 'AddressLine2': 'Folkestone', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT19 6PY', 'Phone': '', 'RatingValue': 4, 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2014-03-31T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.196408', 'latitude': '51.086058'}, 'RightToReply': '', 'Distance': 4591.821311183521, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': Non